## Pairs selection criteria 
### Minimum distance approach as outlined this paper 
#### http://stat.wharton.upenn.edu/~steele/Courses/434/434Context/PairsTrading/PairsTradingGGR.pdf
### To be done after unsupervised clustering techniques have been applied to larger basket
### I will test a cluster of energy assets found from DBSCAN

In [33]:
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import yfinance as yf
import itertools
import math
import random
from numpy import linalg as LA

In [34]:
#Asset list from DBSCAN clusters - utilities
lst=('LNT','AEE','AEP','ATO','CNP','CMS','ED','D','DTE','DUK','EIX','ETR','EVRG','ES','EXC','FE')

#dates for model calibration
start_data='2021-09-01'
end_data='2022-09-01'

#dates for trading with algo
start_trade='2022-08-01'
end_trade='2022-09-01'

index_comp='SPY'

### 1: Fetch returns for training set dates
### 2: Normalize returns and calculate cumulative normalized returns
### 3: Calculate the Sum of Squared Euclidean Distances between the two time series for all possible pair combinations
### 4: Pick pairs with minimum Sum of Squared Euclidean distance

In [35]:
ret_df= pd.DataFrame()
norm_ret_df=pd.DataFrame()
for j in range(0,len(lst)):
    X = yf.download(lst[j], start = start_data, end=end_data)
    X.reset_index(inplace=True)
    X.tail()
    X['Log_Returns'] = np.log(X['Adj Close']) - np.log(X['Adj Close'].shift(1))
    x = X['Log_Returns'].drop(labels=0, axis=0)
    date=X['Date'].drop(labels=0,axis=0)
    date=date.to_frame()

    x_df=x.to_frame().dropna()

    b=x_df.rename(columns={'Log_Returns': lst[j]})
    mu=b[lst[j]].mean()
    std=b[lst[j]].std()
    b=b[lst[j]].values
    ret_df[lst[j]]=b
    elements=[]

    for i in range (0,len(b)):
        element=(b[i]-mu)/std
        elements.append(element)
    norm_ret_df[j] = elements
    norm_ret_df=norm_ret_df.rename(columns={j: lst[j]})  

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [36]:
norm_ret_df

,LNT,AEE,AEP,ATO,CNP,CMS,ED,D,DTE,DUK,EIX,ETR,EVRG,ES,EXC,FE
0,0.500417,0.188746,0.349633,-0.012687,0.700844,0.667232,0.640529,0.542352,0.512977,0.421583,1.025863,0.735511,0.390822,0.800964,0.667199,0.963407
1,-0.722858,-0.864814,-0.522340,-0.686389,0.429300,-0.820149,-0.972518,-0.555695,-1.032625,-0.818040,-0.636370,-0.750992,-0.559737,-0.528234,-0.485391,-0.669182
2,-2.172779,-1.423663,-1.437551,-1.150446,-0.483613,-1.820409,-1.621359,-1.997179,-1.110296,-2.100827,-0.079609,-0.055839,-1.579103,-1.919489,-0.532357,-2.281648
3,1.675225,2.223910,1.595882,1.165405,1.881091,1.481507,2.097797,1.383907,1.460924,1.777902,0.501822,1.462163,1.446691,1.831377,1.182665,0.947004
4,-0.482303,-0.557640,-0.302375,-1.221327,0.187047,-0.368937,-0.665133,0.629928,-0.586648,-0.609666,-0.575974,-1.215148,-0.281016,-1.766963,-0.262783,-1.178688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0.467543,0.494027,0.536802,0.517568,0.164180,0.325683,0.384990,-0.032154,0.310848,-0.198066,0.896925,0.049176,0.346573,0.673313,0.460623,0.627010
247,-1.028918,-1.394681,-1.310586,-1.108847,-1.544346,-1.041736,-0.856467,-1.221071,-1.063549,-1.592379,-0.894250,-0.265652,-1.137073,-1.038580,-1.880070,-1.455321
248,0.275727,0.391352,0.857583,0.195004,0.739774,0.769097,0.468809,0.500471,0.149300,0.362880,0.334712,0.261159,0.816836,0.241507,0.404393,-0.032626
249,-1.175309,-1.758189,-1.340809,-1.350258,-1.122948,-1.573341,-0.841828,-1.280958,-1.393535,-1.246548,-0.909941,-1.593570,-0.947697,-0.947403,-1.628438,-0.903337


In [43]:
cumulative_norm_ret=norm_ret_df.cumsum()
cumulative_norm_ret

,LNT,AEE,AEP,ATO,CNP,CMS,ED,D,DTE,DUK,EIX,ETR,EVRG,ES,EXC,FE
0,5.004172e-01,1.887455e-01,3.496334e-01,-1.268699e-02,7.008445e-01,6.672320e-01,6.405289e-01,5.423517e-01,5.129769e-01,4.215825e-01,1.025863e+00,7.355106e-01,3.908220e-01,8.009645e-01,6.671989e-01,9.634070e-01
1,-2.224410e-01,-6.760689e-01,-1.727063e-01,-6.990763e-01,1.130145e+00,-1.529172e-01,-3.319893e-01,-1.334336e-02,-5.196485e-01,-3.964574e-01,3.894939e-01,-1.548139e-02,-1.689154e-01,2.727308e-01,1.818075e-01,2.942252e-01
2,-2.395220e+00,-2.099732e+00,-1.610258e+00,-1.849522e+00,6.465323e-01,-1.973326e+00,-1.953348e+00,-2.010522e+00,-1.629945e+00,-2.497284e+00,3.098844e-01,-7.132003e-02,-1.748019e+00,-1.646758e+00,-3.505492e-01,-1.987422e+00
3,-7.199945e-01,1.241781e-01,-1.437537e-02,-6.841171e-01,2.527624e+00,-4.918197e-01,1.444492e-01,-6.266153e-01,-1.690213e-01,-7.193827e-01,8.117064e-01,1.390843e+00,-3.013280e-01,1.846188e-01,8.321157e-01,-1.040419e+00
4,-1.202298e+00,-4.334614e-01,-3.167508e-01,-1.905445e+00,2.714670e+00,-8.607563e-01,-5.206836e-01,3.312192e-03,-7.556690e-01,-1.329049e+00,2.357323e-01,1.756959e-01,-5.823437e-01,-1.582345e+00,5.693329e-01,-2.219107e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,2.569594e+00,3.448018e+00,2.780443e+00,3.031716e+00,2.288284e+00,2.541510e+00,1.786135e+00,2.564980e+00,3.087256e+00,3.121971e+00,2.620843e+00,2.579191e+00,2.323974e+00,2.669229e+00,3.427741e+00,2.763225e+00
247,1.540677e+00,2.053337e+00,1.469857e+00,1.922870e+00,7.439381e-01,1.499773e+00,9.296674e-01,1.343909e+00,2.023707e+00,1.529593e+00,1.726593e+00,2.313539e+00,1.186901e+00,1.630649e+00,1.547670e+00,1.307905e+00
248,1.816404e+00,2.444689e+00,2.327440e+00,2.117874e+00,1.483712e+00,2.268870e+00,1.398476e+00,1.844380e+00,2.173007e+00,1.892473e+00,2.061305e+00,2.574698e+00,2.003737e+00,1.872156e+00,1.952064e+00,1.275278e+00
249,6.410951e-01,6.865003e-01,9.866316e-01,7.676152e-01,3.607639e-01,6.955296e-01,5.566487e-01,5.634221e-01,7.794717e-01,6.459253e-01,1.151363e+00,9.811278e-01,1.056040e+00,9.247538e-01,3.236258e-01,3.719416e-01


In [45]:
pair_order_list = itertools.combinations(lst,2)
pairs=list(pair_order_list)
asset1_list=[]
asset2_list=[]
euclidean_distance_list=[]
for i in range(0,len(pairs)):
    asset1_list.append(pairs[i][0])
    asset2_list.append(pairs[i][1])

    dist = np.linalg.norm(cumulative_norm_ret[asset1_list[i]]-cumulative_norm_ret[asset2_list[i]])
    euclidean_distance_list.append(dist)

asset1_list,asset2_list




sdd_list=list(zip(pairs,euclidean_distance_list))
sdd_list.sort(key = lambda x: x[1])

In [46]:
sdd1=[]
sdd2=[]
for i in range(0,len(sdd_list)):
    sdd1.append(sdd_list[i][0][0])
    sdd2.append(sdd_list[i][0][1])

selected_stocks = []
selected_pairs = []
opt_asset1=[]
opt_asset2=[]

for i in range(0,len(sdd_list)):
    s1=sdd1[i]
    s2=sdd2[i]

    if (s1 not in selected_stocks) and (s2 not in selected_stocks):
        selected_stocks.append(s1)
        selected_stocks.append(s2)
        pair=s1+' and '+s2
        selected_pairs.append(pair)

    if len(selected_pairs) == math.comb(len(lst),2):
        break

opt_asset1=selected_stocks[0:len(selected_stocks)-1:2]
opt_asset2=selected_stocks[1:len(selected_stocks):2]

print('Pairs with the minimum Sum of Euclidean Squared Distances:',selected_pairs)

['LNT and EVRG', 'AEE and CMS', 'AEP and DTE', 'DUK and ETR', 'CNP and ED', 'ATO and FE', 'EIX and EXC', 'D and ES']

Pairs with the minimum Sum of Euclidean Squared Distances: ['CMS and DUK', 'AEE and ETR', 'EVRG and ES', 'AEP and DTE', 'ED and D', 'CNP and EIX', 'ATO and FE', 'LNT and EXC']


['LNT and EVRG',
 'AEE and CMS',
 'AEP and DTE',
 'DUK and ETR',
 'CNP and ED',
 'ATO and FE',
 'EIX and EXC',
 'D and ES']